# Notebook to develop HMM model to investigate optimal angle selection

## Defining the targets we will be using in our multiaspect classification model

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec

from scipy.special import spherical_jn, spherical_yn

import ipywidgets as widgets
from IPython.display import display

from sklearn.cluster import KMeans
import seaborn as sns
import pandas as pd

## Below is the analytical DWBA solution for a prolate spheroid geometry

In [2]:
def dwba_prolate_spheroid(L, a, g, h, ka, phi):
    """
    DWBA solution for a fluid prolate spheroid.
    Lee, W.-J., Lavery, A. C., and Stanton, T. K. (2012). 
    “Orientation dependence of broadband acoustic backscattering from live squid,” 
    The Journal of the Acoustical Society of America, 131, 4461–4475. doi:10.1121/1.3701876
    
    Parameters
    ----------
    L : float
        Length of spheroid (long axis) [m]
    a : float
        Radius of spheroid (short axis) [m]
    g : float
        Density contrast (rho2/rho1)
    h : float
        Sound speed contrast (c2/c1)
    ka : array_like
        Dimensionless ka (wavenumber * radius)
    phi : float or array_like
        Incident angle [radians]

    Returns
    -------
    fbs : array_like
        Complex backscattering amplitude
    """
    # Convert inputs to arrays and ensure proper broadcasting
    ka = np.atleast_1d(ka)
    phi = np.atleast_1d(phi)

    # Calculate contrast term
    contrast = 1/(g * h**2) + 1/g - 2
    
    # Convert incident angle to beta
    beta = phi[..., np.newaxis] + np.pi / 2  # Add dimension for broadcasting with ka
    # beta = phi + np.pi/2
    
    # Calculate aspect ratio term (L/2a)
    aspect_ratio = L / (2 * a)

    # Calculate argument for Bessel function
    ellip_term = np.sqrt(np.sin(beta)**2 + (aspect_ratio**2) * np.cos(beta)**2)
    bessel_arg = 2 * ka / h * ellip_term
    
    # Calculate spherical Bessel function
    j1 = spherical_jn(1, bessel_arg)
    
    # Calculate final backscattering amplitude
    fbs = (ka**2) * L * contrast/2 * j1/bessel_arg

    return fbs.squeeze()

## Defining the measurement and target constants

In [3]:
a = 0.15 # semi-minor axis (equatorial radius)
g = 1.043 # density contrast
h = 1.053 # sound speed contrast
c = 1500 # speed of sound in water

freq_lowerbound = 100 # frequency response lower bound
freq_upperbound = 30e3 # frequency response upper bound
freq = np.arange(freq_lowerbound, freq_upperbound, 10)
k = 2*np.pi*freq / c # acoustic wavenumber (phase shift per meter)
ka = k*a

In [ ]:
# BUILD TRAINING DATASET USING DISTORED WAVE BORN APPROXIMATION FROM PROLATE SPHEROID OF ASPECT RATIO = 1
trained_target_AR = 1.5
last_state = 90
first_state = -90

measurement_angle_step = 0.5
angle_all = np.arange(first_state, last_state+measurement_angle_step, measurement_angle_step)
L = trained_target_AR * 2*a
phi_all = np.deg2rad(angle_all)
fbs_full = dwba_prolate_spheroid(L, a, g, h, ka, phi_all)
TS_standard = 20*np.log10(np.abs(fbs_full))

# PERFORM VECTOR QUANTIZATION ON WAVEFORMS MEASURED FROM TRAINING TARGET TO BUILD CODEBOOK
k = TS_standard.shape[0]//20
state = 1500
kmean_TS_codebook = KMeans(n_clusters=k, n_init=10, random_state=state).fit(TS_standard)
labels = kmean_TS_codebook.labels_

## DEFINE STATE CENTERS
state_angle_step = 5
state_centers = np.arange(-last_state, last_state, state_angle_step) + (state_angle_step / 2)

## DEVELOP EMISSION MATRIX FOR TRAINED MODEL
emission_matrix_B = np.zeros((state_centers.shape[0], kmean_TS_codebook.n_clusters))
for i, state_center in enumerate(state_centers):
    angle_indices_for_state = np.where((state_center-(state_angle_step/2)<=angle_all)&(angle_all<state_center+(state_angle_step/2)))[0]
    TS_state_target_k = TS_standard[angle_indices_for_state,:]
    target_k_state_cluster_preds = kmean_TS_codebook.predict(TS_state_target_k)

    density, clusters = np.histogram(target_k_state_cluster_preds, bins=np.arange(0, kmean_TS_codebook.n_clusters+1), density=True)
    emission_matrix_B[i, :] = density

## DEFINE (UNKNOWN) INITIAL STATE AND (KNOWN) PROBABILITY OF BEING IN ANY INITIAL STATE
initial_state_prob = state_angle_step / (last_state - first_state)
current_state = 0
true_target_AR = 2.0
true_target_L = true_target_AR * 2*a

## GENERATE RECEIVED WAVEFORM AND OBSERVATION FROM CURRENT INITIAL STATE
phi = np.deg2rad(current_state)
fbs = dwba_prolate_spheroid(true_target_L, a, g, h, ka, phi)
received_TS = 20*np.log10(np.abs(fbs))
received_code = kmean_TS_codebook.predict(received_TS.reshape((1, freq.shape[0])))[0]

## COMPUTE LIKELIHOOD OF RECEIVING OBSERVATION USING EMISSION MATRIX
alpha_states_for_received_code = initial_state_prob * emission_matrix_B[:,received_code]
likelihood_of_received_code = alpha_states_for_received_code.sum(axis=0)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.2       , 0.28571429, 0.28571429, 0.22857143,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [34]:
def w_(theta):
    sigma_i = state_angle_step / 2
    normalization = 1 / np.sqrt(2*np.pi*(sigma_i**2))

    return normalization * np.exp(-0.5*((theta/sigma_i)**2))

def state_transition_model(delta_angle):
    angular_dist_between_states = (state_centers[np.newaxis, :] - state_centers[:, np.newaxis])

    return w_(angular_dist_between_states - delta_angle)

In [35]:
candidate_angle_steps = np.array([-5, 5])
forecasted_expected_log_likelihood_per_trial = np.zeros(candidate_angle_steps.shape[0])

for a_step, candidate_angle_step in enumerate(candidate_angle_steps):
    A_step = state_transition_model(candidate_angle_step)
    A_step = A_step / A_step.sum(axis=1, keepdims=True)

    likelihood_observing_and_transitioning = np.matmul(alpha_states_for_received_code, A_step)
    alpha_targets_for_forecasted_codes = likelihood_observing_and_transitioning[:, np.newaxis] * emission_matrix_B

    likelihood_for_forecasted_codes = alpha_targets_for_forecasted_codes.sum(axis=0)
    numerator = likelihood_for_forecasted_codes
    denominator = likelihood_of_received_code.sum()

    log_likelihood_targets_for_forecasted_codes = np.log(np.clip(likelihood_for_forecasted_codes, 1e-12, None))
    expected_log_likelihood_of_forecasted_belief = np.sum(log_likelihood_targets_for_forecasted_codes * numerator / denominator)

    forecasted_expected_log_likelihood_per_trial[a_step] = expected_log_likelihood_of_forecasted_belief

angle_step_to_make = candidate_angle_steps[(forecasted_expected_log_likelihood_per_trial).argmax()]
current_state = (current_state + angle_step_to_make)
print(f'Moving to: {current_state}')

Moving to: 5
